In [19]:
import os

In [30]:
%pwd

'c:\\Users\\Subil Varghese Jacob\\OneDrive\\Desktop\\Books\\Oil and gas'

In [21]:
os.chdir("../")

In [22]:
%pwd

'c:\\Users\\Subil Varghese Jacob\\OneDrive\\Desktop\\Books\\Oil and gas'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name : str
    target_column: str


In [27]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH,
                 param_filepath = PARAM_FILE_PATH):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAM_FILE_PATH)
        self.schema=read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_roots])
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        schema=self.schema.TARGET_COLUMNS

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name
        )

        return model_trainer_config

In [28]:
from mlProject import logger
from sklearn.linear_model import LinearRegression
import joblib
import pandas as pd


class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data=pd.read_csv(self.config.test_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data([self.config.target_column])
        test_y=test_data([self.config.target_column])

        lr=LinearRegression()
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
            





In [29]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e 


FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'